In [ ]:
#week-2 Abdul Ghaffar Assignment-02

In [1]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
def readTable(tableName):
    """ Used to built connection with database and read table data 
    Args:
        tableName (str): table name to read data from
    Returns:
        dataframe_mysql: mysql dataframe
    """
    dataframe_mysql = spark.read.format("jdbc").options(
        url="jdbc:mysql://localhost:3306/pyspark",
        driver = "com.mysql.jdbc.Driver",
        dbtable = tableName,
        user="ghaffar",
        password="1234").load()
    return dataframe_mysql

In [4]:
#calling function readTable to read data from the databases
studentTable = readTable('student')
courseTable = readTable('course')

In [5]:
#show table dataframe values
studentTable.show()
courseTable.show()

+---+-------------+-------+
| id|         name|college|
+---+-------------+-------+
|  1|Abdul Ghaffar|   NUST|
+---+-------------+-------+

+---+------------+----------+
| id|        name|student_id|
+---+------------+----------+
| 22|Apache Spark|         1|
+---+------------+----------+



In [6]:
# create temperory view for the give tables to perform sql operations
studentTable.createOrReplaceTempView("students")
studentView = spark.sql("SELECT * FROM students")
studentView.show()

courseTable.createOrReplaceTempView("courses")
courseView = spark.sql("SELECT * FROM courses")
courseView.show()


+---+-------------+-------+
| id|         name|college|
+---+-------------+-------+
|  1|Abdul Ghaffar|   NUST|
+---+-------------+-------+

+---+------------+----------+
| id|        name|student_id|
+---+------------+----------+
| 22|Apache Spark|         1|
+---+------------+----------+



In [7]:
#apply join operation of these two created tables
student_course_join = spark.sql("select students.id as std_id, students.name as std_name, courses.id as course_id, courses.name as course_name  from students join courses on students.id = courses.student_id;")

In [8]:
#show result of new joined table
student_course_join.show()

+------+-------------+---------+------------+
|std_id|     std_name|course_id| course_name|
+------+-------------+---------+------------+
|     1|Abdul Ghaffar|       22|Apache Spark|
+------+-------------+---------+------------+



In [9]:
#save student_course_join table in the mysql database
student_course_join.write.format('jdbc').options(
      url='jdbc:mysql://localhost:3306/pyspark',
      driver='com.mysql.jdbc.Driver',
      dbtable='student_course_join',
      user='ghaffar',
      password='1234').mode('append').save()

In [10]:
#read the saved table again from database
newTable = readTable('student_course_join')
newTable.show()

+------+-------------+---------+------------+
|std_id|     std_name|course_id| course_name|
+------+-------------+---------+------------+
|     1|Abdul Ghaffar|       22|Apache Spark|
|     1|Abdul Ghaffar|       22|Apache Spark|
+------+-------------+---------+------------+

